<a href="https://colab.research.google.com/github/BirukovAlex/neto_Python/blob/main/%D0%94%D0%97_%D1%80%D0%B0%D1%81%D1%85%D0%BE%D0%B4%D1%8B_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предсказываем траты на месяц вперёд



## Загужаем необходимые библиотеки

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Загружаем данные

Собраны данные:
|траты в позапрошлом месяце | траты в прошлом месяце | траты в текущем месяце | траты в следующем месяце|
То есть в таблице должно быть 4 столбца.

8 строчек: 1я строка - итого, 2-8я средние дневные чеки в категориях

In [ ]:
# Загружаем данные из файла Домашняя бухгалтерия (Ответы) - By_Month в переменную df
df=pd.read_csv('Домашняя бухгалтерия (Ответы) - By_Month.csv')


In [ ]:
df.head()

,Current_month,Last_month,Before_last_month,Next_month
0,124857,129850,137659,110000
1,4120,4500,3980,3100
2,12000,15000,14000,10000
3,42000,45000,43400,35000
4,6000,6300,6400,5500


## Смотрим статистику, что нет пропусков и отсуствующих значений

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Current_month      8 non-null      int64
 1   Last_month         8 non-null      int64
 2   Before_last_month  8 non-null      int64
 3   Next_month         8 non-null      int64
dtypes: int64(4)
memory usage: 384.0 bytes


In [ ]:
df.describe()

,Current_month,Last_month,Before_last_month,Next_month
count,8.000000,8.000000,8.000000,8.000000
mean,25847.125000,27856.250000,29567.375000,22325.000000
std,42137.858585,43515.739661,46107.534813,37083.140181
min,1500.000000,1500.000000,1500.000000,1500.000000
25%,3465.000000,4425.000000,3385.000000,2700.000000
50%,9000.000000,10650.000000,10200.000000,7750.000000
75%,21600.000000,23625.000000,31850.000000,17750.000000
max,124857.000000,129850.000000,137659.000000,110000.000000


## Преобразуем строковые данные

In [ ]:
df.head()

,Current_month,Last_month,Before_last_month,Next_month
0,124857,129850,137659,110000
1,4120,4500,3980,3100
2,12000,15000,14000,10000
3,42000,45000,43400,35000
4,6000,6300,6400,5500


## Формируем признаки и целевую переменную

In [ ]:
## Сформируйте признаки и целевую переменную
df.columns

Index(['Current_month', 'Last_month', 'Before_last_month', 'Next_month'], dtype='object')

In [ ]:
X = df[['Current_month', 'Last_month', 'Before_last_month']]
y = df['Next_month']

In [ ]:
X

,Current_month,Last_month,Before_last_month
0,124857,129850,137659
1,4120,4500,3980
2,12000,15000,14000
3,42000,45000,43400
4,6000,6300,6400
5,1500,1500,1500
6,14800,16500,28000
7,1500,4200,1600


In [ ]:
y

0    110000
1      3100
2     10000
3     35000
4      5500
5      1500
6     12000
7      1500
Name: Next_month, dtype: int64

## Разделяем данные на выборку для обучения/проверки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_test

,Current_month,Last_month,Before_last_month
1,4120,4500,3980
4,6000,6300,6400


In [ ]:
X_train

,Current_month,Last_month,Before_last_month
6,14800,16500,28000
7,1500,4200,1600
5,1500,1500,1500
2,12000,15000,14000
0,124857,129850,137659
3,42000,45000,43400


In [ ]:
y_test

1    3100
4    5500
Name: Next_month, dtype: int64

In [ ]:
y_train

6     12000
7      1500
5      1500
2     10000
0    110000
3     35000
Name: Next_month, dtype: int64

## Создаем и обучаем модель линейной регресии

In [ ]:
# Создайте и обучите модель
Lr=LinearRegression()

In [ ]:
Lr.fit(X_train,y_train)

LinearRegression()

In [ ]:
Lr.coef_

array([ 1.05187835, -0.14715214, -0.0181264 ])

## Получаем предсказание и оцениваем качество

In [ ]:
# Получите предсказание
Lr.predict(X_test)

array([3340.04752162, 5008.83908474])

In [ ]:
## Проверка "на глаз"
pd.DataFrame([y_test.values, Lr.predict(X_test)]).T

,0,1
0,3100.0,3340.047522
1,5500.0,5008.839085


In [ ]:
# Оцените качество, при помощи метода mean_squared_error для тестовой выборки
mean_squared_error(y_test, Lr.predict(X_test))

149430.9286578389

Ошибка ~387 рублей

In [ ]:
mean_squared_error(y_train, Lr.predict(X_train))

596910.5795890129

Ошибка ~773 рубля. отличие х2 обусловлено тем, что в выборку для обучения попали данные итоговых трат.

In [ ]:
## Предсказание по всему массиву
pd.DataFrame([y.values, Lr.predict(X.values)]).T

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


,0,1
0,110000.0,109472.044488
1,3100.0,3340.047522
2,10000.0,9902.124956
3,35000.0,36510.995194
4,5500.0,5008.839085
5,1500.0,1070.536126
6,12000.0,12372.886516
7,1500.0,671.412720
